# Assignment 8: Data Aggregation and Group Operations

## Overview
This assignment covers data aggregation and group operations using healthcare/insurance EHR data. You'll analyze patient encounters (insurance claims), healthcare providers, and medical facilities across different regions. The data includes provider statistics, facility information, and encounter charges with insurance coverage details.

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set random seed for reproducibility
np.random.seed(42)

# Create output directory
os.makedirs('output', exist_ok=True)

## Question 1: Basic GroupBy Operations

### Part 1.1: Load and Explore Data

**TODO: Load and merge the datasets**

In [2]:
# TODO: Load the three CSV files into dataframes
provider_df  = pd.read_csv('data/provider_data.csv')
facility_df  = pd.read_csv('data/facility_data.csv')
encounter_df = pd.read_csv('data/encounter_data.csv')

# TODO: Print the shapes of each dataframe
print("Provider data shape:", provider_df.shape)
print("Facility data shape:", facility_df.shape)
print("Encounter data shape:", encounter_df.shape)

# TODO: Merge the dataframes for analysis
merged_df = (
    encounter_df
    .merge(provider_df, on='provider_id', how='left', suffixes=('', '_prov'))
    .merge(facility_df, on='facility_id', how='left', suffixes=('', '_fac'))
)
if 'region_fac' in merged_df.columns:
    merged_df['region'] = merged_df['region'].fillna(merged_df['region_fac'])
    merged_df.drop(columns=['region_fac'], inplace=True, errors='ignore')

# Hint: Start by merging encounter_df with provider_df on 'provider_id'
#       Then merge the result with facility_df on 'facility_id'
# Note: Both encounter_df and provider_df have 'facility_id', and both encounter_df and facility_df have 'region'
#       After merging, you may need to handle duplicate column names (pandas will add suffixes like _x and _y)
#       You can drop one of the duplicate columns and rename if needed

# TODO: Print the merged dataframe shape, columns, and first few rows
print("\nMerged data shape:", merged_df.shape)
print("\nColumns:", list(merged_df.columns))
print("\nFirst few rows:")
print(merged_df.head())

Provider data shape: (500, 7)
Facility data shape: (10, 6)
Encounter data shape: (5000, 12)

Merged data shape: (5000, 22)

Columns: ['encounter_id', 'patient_id', 'provider_id', 'facility_id', 'encounter_date', 'encounter_type', 'diagnosis_code', 'procedure_code', 'service_charge', 'insurance_paid', 'patient_paid', 'region', 'provider_name', 'provider_type', 'facility_id_prov', 'specialty', 'years_experience', 'license_number', 'facility_name', 'facility_type', 'beds', 'established_date']

First few rows:
  encounter_id patient_id provider_id facility_id encounter_date  \
0     ENC00001   PAT01564      PR0259      FAC004     2023-08-21   
1     ENC00002   PAT00296      PR0258      FAC001     2023-09-11   
2     ENC00003   PAT01381      PR0213      FAC001     2023-03-20   
3     ENC00004   PAT01131      PR0146      FAC004     2023-09-03   
4     ENC00005   PAT01388      PR0354      FAC003     2023-06-14   

         encounter_type diagnosis_code  procedure_code  service_charge  \
0    

### Part 1.2: Basic Aggregation

**TODO: Perform basic groupby operations**

In [ ]:
# TODO: Group by facility (healthcare facility) and calculate basic stats
prov_by_fac = (
    provider_df.groupby('facility_id')
    .agg(
        total_providers=('provider_id', 'nunique'),
        avg_years_experience=('years_experience', 'mean'),
        years_exp_sum=('years_experience', 'sum'),
        years_exp_count=('years_experience', 'count')
    )
    .reset_index()
)

enc_finance = (
    encounter_df.groupby('facility_id')
    .agg(
        total_service_charge=('service_charge', 'sum'),
        total_insurance_paid=('insurance_paid', 'sum')
    )
    .reset_index()
)

fac_names = facility_df[['facility_id', 'facility_name']]

q1_summary = (
    fac_names
    .merge(prov_by_fac, on='facility_id', how='left')
    .merge(enc_finance, on='facility_id', how='left')
    .loc[:, ['facility_id', 'facility_name', 'total_providers',
             'avg_years_experience', 'total_service_charge', 'total_insurance_paid']]
    .fillna({'total_providers': 0, 'avg_years_experience': 0.0,
             'total_service_charge': 0.0, 'total_insurance_paid': 0.0})
    .sort_values('total_service_charge', ascending=False)
)
q1_summary.to_csv('output/q1_groupby_analysis.csv', index=False)
print("Saved output/q1_groupby_analysis.csv")

if len(q1_summary):
    top_fac = q1_summary.iloc[0]
    print("\nTop facility by total charges:")
    print(top_fac.to_string(index=True))
else:
    print("No facilities found.")


Saved output/q1_groupby_analysis.csv

Top facility by total charges:
facility_id                            FAC007
facility_name           Pediatric Care Center
total_providers                            48
avg_years_experience                19.854167
total_service_charge                361811.04
total_insurance_paid                291790.95


### Part 1.3: Transform Operations

**TODO: Use transform operations to add group statistics**

In [4]:
# TODO: Add facility mean years_experience as new column
# TODO: Add facility standard deviation of years_experience
# TODO: Create normalized years_experience (z-score within facility)
# TODO: Add facility total encounter charges as new column

# TODO: Display the enhanced dataframe
# TODO: Save results as 'output/q1_aggregation_report.txt'

prov_t = provider_df.copy()

prov_t['fac_mean_years_exp'] = (
    prov_t.groupby('facility_id')['years_experience'].transform('mean')
)
prov_t['fac_std_years_exp'] = (
    prov_t.groupby('facility_id')['years_experience'].transform('std')
)

std = prov_t['fac_std_years_exp'].replace(0, np.nan)
prov_t['years_exp_z_within_fac'] = (prov_t['years_experience'] - prov_t['fac_mean_years_exp']) / std
prov_t['years_exp_z_within_fac'] = prov_t['years_exp_z_within_fac'].fillna(0)

fac_total_charge = encounter_df.groupby('facility_id')['service_charge'].sum()
prov_t['facility_total_service_charge'] = prov_t['facility_id'].map(fac_total_charge).fillna(0)

prov_t = prov_t.merge(
    facility_df[['facility_id', 'facility_name']], on='facility_id', how='left'
)

display_cols = [
    'provider_id','provider_name','facility_id','facility_name','years_experience',
    'fac_mean_years_exp','fac_std_years_exp','years_exp_z_within_fac',
    'facility_total_service_charge'
]
print(prov_t[display_cols].head(10))

report = []
report.append("=== Q1 Aggregation Report ===")
report.append(f"Total providers: {prov_t['provider_id'].nunique():,}")
report.append(f"Total facilities: {prov_t['facility_id'].nunique():,}")
report.append("")

per_fac = (
    prov_t.groupby(['facility_id','facility_name'])
    .agg(
        providers=('provider_id','nunique'),
        years_exp_mean=('years_experience','mean'),
        years_exp_std=('years_experience','std'),
        facility_total_service_charge=('facility_total_service_charge','first')
    )
    .sort_values('facility_total_service_charge', ascending=False)
)

report.append("Per-facility summary (top 15 by total service charge):")
report.append(per_fac.head(15).to_string())

report.append("\nSample transformed provider rows (head 10):")
report.append(prov_t[display_cols].head(10).to_string(index=False))

with open('output/q1_aggregation_report.txt', 'w') as f:
    f.write("\n".join(report))

print("Saved output/q1_aggregation_report.txt")

  provider_id     provider_name facility_id              facility_name  \
0      PR0001        Sam Wilson      FAC006  Oncology Treatment Center   
1      PR0002    Quinn Anderson      FAC001      City General Hospital   
2      PR0003   Taylor Anderson      FAC010  Surgical Specialty Center   
3      PR0004       Jamie Brown      FAC006  Oncology Treatment Center   
4      PR0005  Cameron Williams      FAC001      City General Hospital   
5      PR0006    Riley Williams      FAC006  Oncology Treatment Center   
6      PR0007       Jamie Jones      FAC010  Surgical Specialty Center   
7      PR0008     Cameron Davis      FAC008   Emergency Medical Center   
8      PR0009      Jordan Brown      FAC002    Community Health Clinic   
9      PR0010  Cameron Anderson      FAC002    Community Health Clinic   

   years_experience  fac_mean_years_exp  fac_std_years_exp  \
0                20           21.886792           9.658915   
1                17           23.307692          10.888991   

## Question 2: Advanced GroupBy Operations

### Part 2.1: Filter Operations

**TODO: Use filter operations to remove groups**

In [ ]:
# TODO: Filter facilities with more than 20 providers (sufficient staffing)
# TODO: Filter facilities with average years_experience > 10 (experienced staff)
# TODO: Filter facilities with total encounter charges > 250000 (high volume)

# TODO: Create a summary of filtered results
# TODO: Save results as 'output/q2_filter_analysis.csv'

### Part 2.2: Apply Operations

**TODO: Use apply operations with custom functions**

In [ ]:
# TODO: Create custom function to calculate provider statistics
def provider_stats(group):
    # TODO: Return mean, std, min, max, range for years_experience
    pass

# TODO: Apply custom function to each facility
# Note: Add include_groups=False to avoid pandas FutureWarning about groupby columns
facility_provider_stats = merged_df.groupby('facility_name').apply(provider_stats, include_groups=False)

# TODO: Create function to find top providers in each facility
def top_providers(group, n=2):
    """Return top n providers in a group by encounter volume"""
    # TODO: Return top n rows by encounter count, selecting only provider_id, provider_name, 
    #       years_experience, specialty
    pass

# TODO: Apply function to get top 2 providers per facility
# IMPORTANT: When using groupby().apply(), the result has a MultiIndex where:
# - Level 0: The grouping column (facility_name)
# - Level 1: The original row index from each group
# Use reset_index(level=0) to move only the grouping column back to a regular column
# DO NOT use reset_index(drop=True) - that would drop ALL index levels including facility_name!
top_providers_by_facility = merged_df.groupby('facility_name').apply(
    lambda x: top_providers(x, n=2)
).reset_index(level=0)  # Moves facility_name from index to column

# TODO: Save results as 'output/q2_performance_report.txt'

### Part 2.3: Hierarchical Grouping

**TODO: Perform multi-level grouping**

In [ ]:
# TODO: Group by facility type and region
# TODO: Calculate statistics for each facility_type-region combination
# TODO: Use unstack to convert to wide format
# TODO: Use stack to convert back to long format

# TODO: Analyze the hierarchical structure
# TODO: Save results as 'output/q2_hierarchical_analysis.csv'

## Question 3: Pivot Tables and Cross-Tabulations

### Part 3.1: Basic Pivot Tables

**TODO: Create pivot tables for multi-dimensional analysis**

In [ ]:
# TODO: Create pivot table: encounter charges by procedure code and region
# TODO: Create pivot table with multiple aggregations (sum, mean, count)
# TODO: Add totals (margins) to pivot table
# TODO: Handle missing values with fill_value

# Tip: When saving a pivot table to CSV, the index (row labels) is preserved by default.
#      This is usually desired - when reading back, use index_col=0 to restore the index.
# TODO: Save results as 'output/q3_pivot_analysis.csv'

### Part 3.2: Cross-Tabulations

**TODO: Create cross-tabulations for categorical analysis**

In [ ]:
# TODO: Create crosstab of facility type vs region with counts
# TODO: Create crosstab with margins (row and column totals)
# TODO: Create multi-dimensional crosstab showing facility type × encounter type × region combinations
#       (Hint: use pd.crosstab with multiple index/column parameters, or create manually from the data)

# TODO: Analyze the cross-tabulation results
# TODO: Save results as 'output/q3_crosstab_analysis.csv'

### Part 3.3: Pivot Table Visualization

**TODO: Create visualizations from pivot tables**

In [ ]:
# TODO: Create heatmap from pivot table
# TODO: Create bar chart from pivot table
# TODO: Customize colors and styling
# TODO: Add appropriate titles and labels

# TODO: Save the plot as 'output/q3_pivot_visualization.png'

## Submission Checklist

Before submitting, verify you've created:

- [ ] `output/q1_groupby_analysis.csv` - Basic groupby analysis
- [ ] `output/q1_aggregation_report.txt` - Aggregation report
- [ ] `output/q2_filter_analysis.csv` - Filter operations analysis
- [ ] `output/q2_hierarchical_analysis.csv` - Hierarchical analysis
- [ ] `output/q2_performance_report.txt` - Performance report
- [ ] `output/q3_pivot_analysis.csv` - Pivot table analysis
- [ ] `output/q3_crosstab_analysis.csv` - Cross-tabulation analysis
- [ ] `output/q3_pivot_visualization.png` - Pivot visualization

## Key Learning Objectives

- Master the split-apply-combine paradigm
- Apply aggregation functions and transformations
- Create pivot tables for multi-dimensional analysis
- Apply advanced groupby techniques